### Importação das bibliotecas e autenticação da GEE API

In [1]:
import ee
import geemap

try:
    ee.Initialize(project = 'teste-476811')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'teste-476811')

### Criação do mapa interativo

In [19]:
Map = geemap.Map(center=(-5, -50), zoom=5)
Map

Map(center=[-5, -50], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tra…